In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from src.correction import load_words_from_file, custom_tokenizer
import pandas as pd
import dill
import os

Загрузка обучающих данных

In [2]:
train_file_path = "./input/payments_main_predictions_2_models_gree_valid_v2.tsv"
df_train = pd.read_csv(train_file_path, sep="\t", header=None)
df_train.columns = [
    "id", "date", "cost", "text", "category_gemma_2b", "prob_gemma_2b",
    "category_gemma_9b", "prob_gemma_9b", "category",
]

In [3]:
dictionary = load_words_from_file("./src/dictionary/russian.txt")
df_train["text"] = df_train["text"].apply(lambda x: custom_tokenizer(x, dictionary))

2024-11-17 10:55:23,176 INFO:Исправлено 'октбяря' на 'октября' (перестановка двух букв)
2024-11-17 10:55:23,180 INFO:Исправлено 'договроу' на 'договору' (перестановка двух букв)
2024-11-17 10:55:23,183 INFO:Исправлено 'телек' на 'телке' (перестановка двух букв)
2024-11-17 10:55:23,186 INFO:Исправлено 'средтсв' на 'средств' (перестановка двух букв)
2024-11-17 10:55:23,187 INFO:Исправлено 'лизигна' на 'лизинга' (перестановка двух букв)
2024-11-17 10:55:23,195 INFO:Исправлено 'декбаря' на 'декабря' (перестановка двух букв)
2024-11-17 10:55:23,199 INFO:Исправлено 'госитницу' на 'гостиницу' (перестановка двух букв)
2024-11-17 10:55:23,203 INFO:Исправлено 'телек' на 'телке' (перестановка двух букв)
2024-11-17 10:55:23,205 INFO:Исправлено 'аперля' на 'апреля' (перестановка двух букв)
2024-11-17 10:55:23,210 INFO:Исправлено 'доогвору' на 'договору' (перестановка двух букв)
2024-11-17 10:55:23,221 INFO:Исправлено 'тоавров' на 'товаров' (перестановка двух букв)
2024-11-17 10:55:23,221 INFO:Испра

Загрузка тестовых данных

In [4]:
test_file_path = "./input/payments_training.tsv"
df_test = pd.read_csv(test_file_path, sep="\t", header=None)
df_test.columns = ["index", "date", "amount", "text", "category"]

In [5]:
df_test["text"] = df_test["text"].apply(lambda x: custom_tokenizer(x, dictionary))

2024-11-17 10:55:25,097 INFO:Исправлено 'телек' на 'телке' (перестановка двух букв)


Инициализация TfidfVectorizer

In [6]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_train["text"])
y = df_train["category"]

Инициализация классификатора Ridge

In [7]:
model = RidgeClassifier(alpha=0)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1)
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean Accuracy: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")

Cross-Validation Accuracy Scores: [1.     1.     1.     1.     0.9996 1.     1.     1.     1.     1.    ]
Mean Accuracy: 1.0000
Standard Deviation: 0.0001


Обучение модели на всех доступных обучающих данных

In [8]:
model.fit(X, y)
X_test = vectorizer.transform(df_test["text"])
y_test = df_test["category"]
y_pred = model.predict(X_test)

Вычисление точности на тестовом наборе

In [9]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Final Test Accuracy: {accuracy:.4f}")
print("Classification Report:", report, sep="\n")

Final Test Accuracy: 1.0000
Classification Report:
                precision    recall  f1-score   support

  BANK_SERVICE       1.00      1.00      1.00        49
    FOOD_GOODS       1.00      1.00      1.00        90
       LEASING       1.00      1.00      1.00        38
          LOAN       1.00      1.00      1.00        41
NON_FOOD_GOODS       1.00      1.00      1.00        96
NOT_CLASSIFIED       1.00      1.00      1.00        23
   REALE_STATE       1.00      1.00      1.00        27
       SERVICE       1.00      1.00      1.00        88
           TAX       1.00      1.00      1.00        48

      accuracy                           1.00       500
     macro avg       1.00      1.00      1.00       500
  weighted avg       1.00      1.00      1.00       500



Сохранение векторизатора и классификатора в файл с использованием dill

In [10]:
os.system("rm -rf ./src/models; mkdir ./src/models")
with open("./src/models/tf-idf-vectorizer.dill", "wb") as f:
    dill.dump(vectorizer, f)
with open("./src/models/classifier.dill", "wb") as f:
    dill.dump(model, f)